# Datenanalyse

Laden

ansicht

In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import csv
csv_path = '../data/roh/airscout_full-spectrum_neustadt_weinstrasse_2025_07190605.csv'
# Finde die erste Zeile ohne '#' als Header
with open(csv_path, encoding='utf-8') as f:
    for i, row in enumerate(f):
        if not row.strip().startswith('#'):
            header_row = i
            break
# Versuche zuerst mit Komma, dann mit Semikolon als Trennzeichen zu laden
try:
    df = pd.read_csv(csv_path, skiprows=header_row-1)
    if df.shape[1] == 1:
        raise ValueError('Nur eine Spalte erkannt, versuche sep=;')
except Exception:
    df = pd.read_csv(csv_path, skiprows=header_row-1, sep=';')
df.reset_index(drop=True, inplace=True)
print('Spaltennamen:', list(df.columns))

Spaltennamen: ['# ===================================']


In [31]:
# Entferne alle Zeit-Zusätze wie ' MESZ', ' UTC', ' utc', 'UTC1', 'UT', 'CET' usw. am Ende der Strings
import re
for col in ['DateTime', 'GPS_DateTime']:
    if col in df.columns:
        df[col] = df[col].str.replace(r'\s*(MESZ|mesz|UTC\d*|utc\d*|UT\d*|ut\d*|CET|cet)$', '', regex=True)

In [32]:

display(df.head())
df.describe()


,# ===================================
0,"SecSinceMidnight-MS,Temperature_DHT_C,Humidity..."
1,"21955-985,0.0,0.0,7,99.3,--,--,--,--,--,0,16,1..."
2,"21957-982,25.2,47.0,7,99.5,--,--,--,--,--,0,19..."
3,"21959-060,25.3,47.0,7,99.3,--,--,--,--,--,0,21..."
4,"21961-355,25.3,47.0,8,99.3,--,--,--,--,--,0,27..."


,# ===================================
count,1947
unique,1947
top,"SecSinceMidnight-MS,Temperature_DHT_C,Humidity..."
freq,1
